In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
from scipy.stats import norm
#%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

class Clock_Animation: #класс для рисования
    def __init__(self, U, psi):
        self.U = U
        self.psi = psi
        self.fig = plt.figure()
        self.Nx = psi.shape[1]
        self.ax = plt.axes(xlim=(0, self.Nx), ylim=(-.002, .002))
        self.line_density, = self.ax.plot([], [], lw=2)
        self.line_potential, = self.ax.plot([], [], lw=2)
        
        #Для корректной работы установить ffmpeg 
        #Для среды notebook вбить в anaconda: conda install -c conda-forge ffmpeg
        anim = animation.FuncAnimation(self.fig, self.animate, init_func=self.initialization_func, 
                                       frames=2000, interval=20, blit=True)
        anim.save('Clock.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
        plt.show()
    
    #initialization function: plot the background of each frame
    def initialization_func(self):
        self.line_density.set_data([], [])
        self.line_potential.set_data([], [])
        return self.line_density, self.line_potential
    
    # animation function.  This is called sequentially
    def animate(self, i):
        A = self.psi[i % (self.psi.shape[0])] # '%' для зацикливания анимации
        
        x = np.arange(0, self.Nx)
        y = A*np.conjugate(A)
        
        self.line_density.set_data(x, y)
        self.line_potential.set_data(np.arange(0, self.Nx), U)
        return self.line_density, self.line_potential

    
class Clock:
    def __init__(self, Lt, U, psi_profile, Tmax):
        self.hx = 1./(U.size - 1) #шаг по координате
        self.Lt = Lt #шаг по времени
                
        self.U = U
        
        self.Nx = int(U.size)
        self.Nt = int(Tmax/Lt + 1)
        self.psi = np.zeros(shape=(self.Nt, self.Nx), dtype = complex)
        
              
        #Считаем, что на границах в.ф. точный ноль всегда
        self.psi[:, 0] = 0. 
        self.psi[:,-1] = 0.
        
        self.psi[0] = psi_profile #начальное распределение
        
        #Вместо первого шага по явной схеме скопируем исходный профиль
        self.psi[1] = psi_profile
        
        self.Dufert_Frankel_scheme()
               
        clock_anim = Clock_Animation(U0, self.psi)
        
    def Dufert_Frankel_scheme(self):
        for i in range(1, self.Nt - 1):
            self.psi[i+1, 1:self.Nx-1] =\
                (self.psi[i, 2:self.Nx]/self.hx**2 + self.psi[i, 0:self.Nx-2]/self.hx**2 -\
                 self.psi[i-1, 1:self.Nx-1]*(1./self.hx**2 + self.U[1:self.Nx-1]/2. + 1j/(2.*self.Lt)))*\
                 (1/self.hx**2 + self.U[1:self.Nx-1]/2. - 1j/(2.*self.Lt))**(-1)
                    

Lt = 0.01 #шаг по времени
Time_steps = 10

#Профиль U
length = 101
width_hole = 40
width_barrier = 2

U0 = -10.
#U = np.zeros(length)
U = np.ones(length)*1000
U[ (length - width_barrier)/2 - width_hole : (length - width_barrier)/2 ] = U0
U[ (length + width_barrier)/2 : (length + width_barrier)/2 + width_hole ] = U0
#print U


#гауссов пакет в левой яме
sigma = 10.
mu = (length - width_barrier - width_hole)/2.  
psi_profile = np.zeros(length)
psi_profile[ (length - width_barrier)/2 - width_hole : (length - width_barrier)/2] =\
    norm(mu, sigma).pdf(np.arange((length - width_barrier)/2 - width_hole, (length - width_barrier)/2)) 


#print psi_profile
Clock(Lt, U, psi_profile, Time_steps)
print 'total probability = ',\
    psi_profile[ (length - width_barrier)/2 - width_hole : (length - width_barrier)/2].sum()

total probability =  0.953509425991
